<a href="https://colab.research.google.com/github/fzantalis/colab_collection/blob/master/first_order_model_ttmai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"

**Clone repository and install all the requirments**

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

SyntaxError: ignored

**Restart Runtime**

Επιλέγουμε από το μενού (Runtime -> Restart Runtime).

Ή πατάμε τα πλήκτρα συντόμευσης ( Ctrl + M + . )   (ναι.. πατάμε και την τελεία)

In [0]:
cd first-order-model

**Mount your Google drive folder on Colab**

Στην συνέχεια χρειαζόμαστε τα αρχεία από το εκπαιδευμένο μοντέλο. Τα αρχεία αυτά είναι μεγάλα σε όγκο, οπότε συνδέσουμε το google drive μας με το colab και θα προσθέσουμε στο drive μας έναν κοινόχρηστο φάκελο που μας προσφέρει ο developer αυτής της εφαρμογής.

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

**Load driving video and source image**

In [0]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")


**Create a model and load checkpoints**

In [0]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

**Perfrorm image animation**

In [0]:
from demo import make_animation
from skimage import img_as_ubyte



## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

In [2]:
from google.colab import files 
uploaded = files.upload() 
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

MessageError: ignored

In [0]:
!ffmpeg -i /content/first-order-model/test.mp4 -ss 00:00:00.00 -t 00:00:12 -filter:v "crop=600:600:390:10" -async 1 driving_video.mp4

In [0]:
source_image = imageio.imread('/content/first-order-model/source.png')
driving_video = imageio.mimread('driving_video.mp4', memtest=False)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,  adapt_movement_scale=True)

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video, predictions).to_html5_video())